## FCPXML timing attributes
According to this page: https://developer.apple.com/library/archive/documentation/FinalCutProX/Reference/FinalCutProXXMLFormat/StoryElements/StoryElements.html#//apple_ref/doc/uid/TP40011227-CH13-SW2

Timing values must meet the following requirements:
1. Must be a multiple of the frame duration for the respective timeline
2. Numerator must be 64-bit integer
3. Denomenator must be a 32-bit integer

Frame durations can be either:
1001/30000s (29.97fps), 1001/60000s (59.94fps), or 100/3000s (30fps)

In [106]:
number = 4.0
type(number)
number.is_integer()

True

In [354]:
time_val = 3600456485
if len(str(time_val)) > 16:
    raise ValueError('Time value must be less than 16 digits.')

In [417]:
old_string = "didn't work"
new_string = "worked"

def function():
    _locals = locals()
    exec("old_string = new_string", globals(), _locals)
    exec("new_string = 12", globals(), _locals)
    print(old_string)
    print(_locals)

function()

didn't work
{'old_string': 'worked', 'new_string': 12}


In [188]:
is_int32(0)

True

In [176]:
min32, max32 = -2**31, 2**31-1
min64, max64 = -2**63, 2**63-1
print(f'A 32-bit int is between {min32} and {max32}')
print(f'A 64-bit int is between {min64} and {max64}')

A 32-bit int is between -2147483648 and 2147483647
A 64-bit int is between -9223372036854775808 and 9223372036854775807


## Final functions below

In [439]:
def is_int32(number):
    '''Checks if an integer is 32-bit.'''
    if type(number) != int: return False
    min32, max32 = -2**31, 2**31-1
    if number >= min32 and number <= max32:
        return True
    else: return False
    
def is_int64(number):
    '''Checks if an integer is 64-bit.'''
    if type(number) != int: return False
    min64, max64 = -2**63, 2**63-1
    if number >= min64 and number <= max64:
        return True
    else: return False

In [487]:
def fit2frame(time, frame_durs):
    '''
    Converts a time (int in seconds) into a formatted string which FCPX expects.
    Output is a dict of strings. Each dict key is a frame duration.
    '''
    _locals = locals()
    time_vals = {}
    #if len(str(time)) > 16:
    #    raise ValueError('Time value must be less than 16 digits of precision.')
    for fd in frame_durs.keys():
        num = frame_durs[fd][0]
        den = frame_durs[fd][1]
        exec(f'{fd} = {num} / {den}')
        exec(f'quotient = time / {fd}', globals(), _locals)
        rounded = int(round(_locals['quotient']))
        new_num = int(round(rounded*num))
        check = new_num/num
        if not check.is_integer():
            print(f'Warning, {check} is not an integer!')
        while(is_int64(new_num) == False):
            print(f'Warning, {new_num} is not a 64-bit integer!')
            new_num = int(new_num / 2)
            den = int(den / 2)
        if(is_int32(den) == False):
            print(f'Warning, {den} is not a 32-bit integer!')
        if den == 1:
            string = f'{new_num}s'
        else: string = f'{new_num}/{den}s'
        try: new_time = new_num / den
        except: pass
        time_vals[fd] = string
    return time_vals

In [488]:
def format_timestamp(time, fd):
    '''
    Selects the appropriate timestamp based on the clip's frame duration.
    Output is a string of format "{numerator}/{denomenator}s".
    '''
    fd_refs = {
    "1001/30000s": "fd_2997",
    "1001/60000s": "fd_5994",
    "100/3000s": "fd_30"
    }
    frame_durs = {
    "fd_2997": [1001, 30000],
    "fd_5994": [1001, 60000],
    "fd_30": [100, 3000]
    }
    timestamp = fit2frame(time, frame_durs)[fd_refs[fd]]
    return timestamp

In [489]:
format_timestamp(time=15645645.45644656855, fd="1001/60000s")

468900463 469369363463
468900463.0
937800926 938738726926
937800926.0
469369364 46936936400
469369364.0


'938738726926/60000s'